In [10]:
# Implementation fuzzy system for washing machine 

In [11]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt


In [12]:

dirtiness = np.arange(0, 101, 1)  # Dirtiness level (0 to 100)
fabric_type = np.arange(0, 3, 1)   # Fabric types: 0=Wool, 1=Cotton, 2=Plastic
wash_time = np.arange(0, 61, 1)    # Wash time in minutes (0 to 60)
wash_intensity = np.arange(0, 11, 1)  # Wash intensity (0 to 10)


In [13]:
# 3. Define membership functions
def mf(x, mf_type, *params):
    if mf_type == "trimf":
        return fuzz.trimf(x, list(params))
    elif mf_type == "trapmf":
        return fuzz.trapmf(x, list(params))
    elif mf_type == "gbell":
        return fuzz.gbellmf(x, *params)
    else:
        raise ValueError("Invalid membership function type")
    
    # Dirtiness membership functions
dirt = {
    'x': dirtiness,
    'low': mf(dirtiness, "trimf", 0, 0, 50),
    'medium': mf(dirtiness, "trimf", 0, 50, 100),
    'high': mf(dirtiness, "trimf", 50, 100, 100)
}

# Fabric type membership functions
fabric = {
    'x': fabric_type,
    'wool': mf(fabric_type, "trimf", 0, 0, 1),
    'cotton': mf(fabric_type, "trimf", 0, 1, 2),
    'plastic': mf(fabric_type, "trimf", 1, 2, 2)
}

# Wash time membership functions
time = {
    'x': wash_time,
    'short': mf(wash_time, "trimf", 0, 0, 30),
    'medium': mf(wash_time, "trimf", 0, 30, 60),
    'long': mf(wash_time, "trimf", 30, 60, 60)
}

# Wash intensity membership functions
intensity = {
    'x': wash_intensity,
    'low': mf(wash_intensity, "trimf", 0, 0, 5),
    'medium': mf(wash_intensity, "trimf", 0, 5, 10),
    'high': mf(wash_intensity, "trimf", 5, 10, 10)
}

 

In [14]:

# 4. Plot membership functions
def plotMF():
    plt.figure(figsize=(12, 10))
    
    # Dirtiness plot
    plt.subplot(2, 2, 1)
    plt.plot(dirt['x'], dirt['low'], label='Low')
    plt.plot(dirt['x'], dirt['medium'], label='Medium')
    plt.plot(dirt['x'], dirt['high'], label='High')
    plt.title("Dirtiness Level Membership Functions")
    plt.xlabel("Dirtiness Level")
    plt.ylabel("Membership Degree")
    plt.legend()
    
    # Fabric type plot
    plt.subplot(2, 2, 2)
    plt.plot(fabric['x'], fabric['wool'], label='Wool')
    plt.plot(fabric['x'], fabric['cotton'], label='Cotton')
    plt.plot(fabric['x'], fabric['plastic'], label='Plastic')
    plt.title("Fabric Type Membership Functions")
    plt.xlabel("Fabric Type")
    plt.ylabel("Membership Degree")
    plt.legend()
    
    # Wash time plot
    plt.subplot(2, 2, 3)
    plt.plot(time['x'], time['short'], label='Short')
    plt.plot(time['x'], time['medium'], label='Medium')
    plt.plot(time['x'], time['long'], label='Long')
    plt.title("Wash Time Membership Functions")
    plt.xlabel("Wash Time (minutes)")
    plt.ylabel("Membership Degree")
    plt.legend()
    
    # Wash intensity plot
    plt.subplot(2, 2, 4)
    plt.plot(intensity['x'], intensity['low'], label='Low')
    plt.plot(intensity['x'], intensity['medium'], label='Medium')
    plt.plot(intensity['x'], intensity['high'], label='High')
    plt.title("Wash Intensity Membership Functions")
    plt.xlabel("Wash Intensity")
    plt.ylabel("Membership Degree")
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    plotMF()


In [15]:
# 5. Fuzzy inference system
class FuzzyWashingSystem:
    def __init__(self):
        self.rules = []
    
    def add_rule(self, antecedent, consequent):
        self.rules.append((antecedent, consequent))
    
    def evaluate(self, dirt_input, fabric_input):
        # Fuzzify inputs
        dirt_low = fuzz.interp_membership(dirt['x'], dirt['low'], dirt_input)
        dirt_med = fuzz.interp_membership(dirt['x'], dirt['medium'], dirt_input)
        dirt_high = fuzz.interp_membership(dirt['x'], dirt['high'], dirt_input)
        
        fabric_wool = fuzz.interp_membership(fabric['x'], fabric['wool'], fabric_input)
        fabric_cotton = fuzz.interp_membership(fabric['x'], fabric['cotton'], fabric_input)
        fabric_plastic = fuzz.interp_membership(fabric['x'], fabric['plastic'], fabric_input)
        
        # Initialize output aggregations
        time_agg = np.zeros_like(time['x'])
        intensity_agg = np.zeros_like(intensity['x'])
        
        # Apply rules
        for rule in self.rules:
            antecedent, consequent = rule
            
            # Evaluate antecedent
            if antecedent[0] == 'dirt':
                if antecedent[1] == 'low':
                    mu_dirt = dirt_low
                elif antecedent[1] == 'medium':
                    mu_dirt = dirt_med
                else:
                    mu_dirt = dirt_high
                
                if antecedent[2] == 'fabric':
                    if antecedent[3] == 'wool':
                        mu_fabric = fabric_wool
                    elif antecedent[3] == 'cotton':
                        mu_fabric = fabric_cotton
                    else:
                        mu_fabric = fabric_plastic
                    
                    rule_strength = np.fmin(mu_dirt, mu_fabric)
            
            # Apply to consequent
            if consequent[0] == 'time':
                if consequent[1] == 'short':
                    output = time['short']
                elif consequent[1] == 'medium':
                    output = time['medium']
                else:
                    output = time['long']
                
                time_agg = np.fmax(time_agg, np.fmin(rule_strength, output))
            
            elif consequent[0] == 'intensity':
                if consequent[1] == 'low':
                    output = intensity['low']
                elif consequent[1] == 'medium':
                    output = intensity['medium']
                else:
                    output = intensity['high']
                
                intensity_agg = np.fmax(intensity_agg, np.fmin(rule_strength, output))
        
        # Defuzzify
        time_output = fuzz.defuzz(time['x'], time_agg, 'centroid')
        intensity_output = fuzz.defuzz(intensity['x'], intensity_agg, 'centroid')
        
        return time_output, intensity_output

        
       

In [16]:

# 6. Create and configure fuzzy system
washing_system = FuzzyWashingSystem()


In [17]:

# Add rules
washing_system.add_rule(('dirt', 'low', 'fabric', 'wool'), ('time', 'short'))
washing_system.add_rule(('dirt', 'low', 'fabric', 'wool'), ('intensity', 'low'))
washing_system.add_rule(('dirt', 'medium', 'fabric', 'cotton'), ('time', 'medium'))
washing_system.add_rule(('dirt', 'medium', 'fabric', 'cotton'), ('intensity', 'medium'))
washing_system.add_rule(('dirt', 'high', 'fabric', 'plastic'), ('time', 'long'))
washing_system.add_rule(('dirt', 'high', 'fabric', 'plastic'), ('intensity', 'high'))
washing_system.add_rule(('dirt', 'high', 'fabric', 'cotton'), ('time', 'long'))
washing_system.add_rule(('dirt', 'high', 'fabric', 'cotton'), ('intensity', 'medium'))


In [18]:
# 7. Test the system
dirt_input = 70  # Dirtiness level (0-100)
fabric_input = 1  # 0=Wool, 1=Cotton, 2=Plastic

time_output, intensity_output = washing_system.evaluate(dirt_input, fabric_input)

print("\nFuzzy Washing System Results:")
print(f"Dirtiness level: {dirt_input}")
print(f"Fabric type: {'Wool' if fabric_input == 0 else 'Cotton' if fabric_input == 1 else 'Plastic'}")
print(f"Recommended wash time: {time_output:.1f} minutes")
print(f"Recommended wash intensity: {intensity_output:.1f}")




Fuzzy Washing System Results:
Dirtiness level: 70
Fabric type: Cotton
Recommended wash time: 32.3 minutes
Recommended wash intensity: 5.0
